In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
from google.colab import drive

# ✨ 멀티프로세싱을 위한 모듈 추가
import multiprocessing as mp

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/DL_project')

from dataset.ocr_dataset import OCRDataset
from length_predictor_net import LengthPredictorNet
from common.optimizer import Adam

# --- ✨ 1. 데이터 배치를 미리 생성하는 '생산자' 함수 정의 ---
def data_producer(dataset, queue, batch_size):
    """
    백그라운드 프로세스에서 실행될 함수.
    데이터셋에서 미니배치를 가져와 큐에 넣습니다.
    """
    while True:
        x_batch, t_batch = dataset.get_batch(batch_size)
        if x_batch.shape[0] > 0:
            queue.put((x_batch, t_batch))
# -------------------------------------------------------------

# --- 헬퍼 함수: 텍스트 레이블을 길이 레이블로 변환 (기존과 동일) ---
def convert_to_length_labels(text_labels, max_len, pad_id=0):
    length_labels = []
    for label in text_labels:
        true_len_idx = np.where(label == pad_id)[0]
        if len(true_len_idx) > 0:
            length = true_len_idx[0]
            if length == 0: length = 1
            length_labels.append(length - 1)
        else:
            length_labels.append(max_len - 1)
    return np.array(length_labels, dtype=np.int32)
# ----------------------------------------------------

# 1. 데이터 로드 및 파라미터 설정
print("데이터 로딩 중...")
BASE_GDRIVE_PATH = "/content/drive/MyDrive/DL_project"
TRAIN_IMG_PATH = os.path.join(BASE_GDRIVE_PATH, "train_images")
TRAIN_LBL_PATH = os.path.join(BASE_GDRIVE_PATH, "train_labels")
TEST_IMG_PATH = os.path.join(BASE_GDRIVE_PATH, "test_images")   # ✨ Test 데이터 경로 추가
TEST_LBL_PATH = os.path.join(BASE_GDRIVE_PATH, "test_labels")   # ✨ Test 데이터 경로 추가
VOCAB_PATH = os.path.join(BASE_GDRIVE_PATH, "vocab.json")

IMAGE_WIDTH, IMAGE_HEIGHT = 256, 64
MAX_LABEL_LEN = 25
PAD_ID = 0

# ✨ Train / Test 데이터셋 로드
train_dataset = OCRDataset(TRAIN_IMG_PATH, TRAIN_LBL_PATH, VOCAB_PATH, image_size=(IMAGE_WIDTH, IMAGE_HEIGHT), max_label_len=MAX_LABEL_LEN)
test_dataset = OCRDataset(TEST_IMG_PATH, TEST_LBL_PATH, VOCAB_PATH, image_size=(IMAGE_WIDTH, IMAGE_HEIGHT), max_label_len=MAX_LABEL_LEN)

# 2. 모델, 옵티마이저 생성
network = LengthPredictorNet(input_dim=(1, IMAGE_HEIGHT, IMAGE_WIDTH),
                             max_output_len=MAX_LABEL_LEN)
optimizer = Adam(lr=0.0001)

# 3. 학습 하이퍼파라미터 설정
iters_num = 5000  # ✨ 충분한 학습을 위해 반복 횟수 증가
batch_size = 256
train_size = len(train_dataset.image_files) # ✨ 실제 학습 데이터 크기 사용
iter_per_epoch = max(train_size // batch_size, 1)

train_loss_list = []
train_acc_list = []
test_acc_list = [] # ✨ Test 정확도 리스트 추가

# --- ✨ 2. 멀티프로세싱 설정 ---
NUM_PRODUCERS = mp.cpu_count()
print(f"사용 가능한 CPU 코어 수: {NUM_PRODUCERS}, {NUM_PRODUCERS}개의 생산자 프로세스를 사용합니다.")
data_queue = mp.Queue(maxsize=NUM_PRODUCERS * 2)
producers = []
for _ in range(NUM_PRODUCERS):
    p = mp.Process(target=data_producer,
                   args=(train_dataset, data_queue, batch_size),
                   daemon=True)
    p.start()
    producers.append(p)
# --------------------------------

# ✨ 시간 측정을 위한 변수 초기화
data_time = 0
compute_time = 0

print("\n🚀 글자 수 예측 모델 학습을 시작합니다...")
for i in range(iters_num):
    # --- 데이터 로딩 시간 측정 ---
    start_data = time.time()
    x_batch, t_text_batch = data_queue.get()
    data_time += time.time() - start_data
    # ---------------------------

    # 핵심: 텍스트 레이블을 -> 길이 레이블로 변환
    t_len_batch = convert_to_length_labels(t_text_batch, MAX_LABEL_LEN, PAD_ID)

    # --- 학습 연산 시간 측정 ---
    start_compute = time.time()
    grad = network.gradient(x_batch, t_len_batch)
    optimizer.update(network.params, grad)
    compute_time += time.time() - start_compute
    # ---------------------------

    loss = network.loss(x_batch, t_len_batch)
    train_loss_list.append(loss)

    # ✨ 20회 반복마다 중간 결과 및 시간 출력
    if (i + 1) % 20 == 0:
        avg_data_time = data_time / 20
        avg_compute_time = compute_time / 20
        print(f"Iter: {i+1} / {iters_num} | Loss: {loss:.4f} | "
              f"데이터 로딩: {avg_data_time:.3f}초 | 학습 연산: {avg_compute_time:.3f}초")
        data_time, compute_time = 0, 0 # 시간 변수 초기화

    # ✨ 1 에폭마다 정확도 계산 및 출력
    if (i + 1) % iter_per_epoch == 0:
        epoch_num = (i + 1) // iter_per_epoch

        # Train 데이터 정확도
        x_train_sample, t_train_text_sample = train_dataset.get_batch(100)
        t_train_len_sample = convert_to_length_labels(t_train_text_sample, MAX_LABEL_LEN, PAD_ID)
        train_acc = network.accuracy(x_train_sample, t_train_len_sample)
        train_acc_list.append(train_acc)

        # Test 데이터 정확도
        x_test_sample, t_test_text_sample = test_dataset.get_batch(100)
        t_test_len_sample = convert_to_length_labels(t_test_text_sample, MAX_LABEL_LEN, PAD_ID)
        test_acc = network.accuracy(x_test_sample, t_test_len_sample)
        test_acc_list.append(test_acc)

        print(f"========== EPOCH {int(epoch_num)} ==========")
        print(f"Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        print("==============================")

# --- ✨ 3. 학습 종료 후 생산자 프로세스 정리 ---
for p in producers:
    p.terminate()
# ---------------------------------------------

# 4. 파라미터 저장
params_file = os.path.join(BASE_GDRIVE_PATH, "length_predictor_params.pkl")
with open(params_file, 'wb') as f:
    pickle.dump(network.params, f)
print(f"\n✅ 학습된 파라미터를 '{params_file}'에 저장했습니다.")

# 5. 그래프 그리기
plt.figure(figsize=(12, 5))

# Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(train_loss_list, label='train loss')
plt.xlabel("iterations")
plt.ylabel("loss")
plt.title("Length Predictor: Loss")
plt.legend()

# Accuracy 그래프 (✨ Train / Test 함께 표시)
plt.subplot(1, 2, 2)
plt.plot(train_acc_list, label='train acc')
plt.plot(test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Length Predictor: Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

Mounted at /content/drive
데이터 로딩 중...
🚀 OCR 데이터셋 초기화를 시작합니다...
✅ 단어장 로드 완료. 총 글자 수: 4109
✅ 총 31535개의 이미지 파일을 처리 대상으로 설정했습니다.
🚀 OCR 데이터셋 초기화를 시작합니다...
✅ 단어장 로드 완료. 총 글자 수: 4109
✅ 총 4000개의 이미지 파일을 처리 대상으로 설정했습니다.
사용 가능한 CPU 코어 수: 8, 8개의 생산자 프로세스를 사용합니다.

🚀 글자 수 예측 모델 학습을 시작합니다...
Iter: 20 / 5000 | Loss: 2.0724 | 데이터 로딩: 1.142초 | 학습 연산: 1.956초
Iter: 40 / 5000 | Loss: 1.8427 | 데이터 로딩: 0.016초 | 학습 연산: 1.933초
Iter: 60 / 5000 | Loss: 1.4300 | 데이터 로딩: 0.014초 | 학습 연산: 1.920초
Iter: 80 / 5000 | Loss: 1.6190 | 데이터 로딩: 0.014초 | 학습 연산: 1.907초


KeyboardInterrupt: 